In [385]:
from pprint import pprint
import redis
import json
from pymongo import MongoClient
client = MongoClient('mongodb://localhost:27015')
db = client.abaco

## Conversion Stuff

In [401]:
# Mongo Conversion
# Helper functions that converts Mongo to new formatting
def updateMongoLogs(old_json, executions):
    newList = []
    for actor in old_json:
        aid = actor['_id']
        newDict = {'exp': actor['exp'], '_id': aid, 'logs': actor[aid]}
        newList.append(newDict)
        
    for log in newList:
        wanted_exec_id = log['_id']
        for execution in executions:
            for exec_id, exec_info in execution.items():
                if exec_id == '_id':
                    continue
                if exec_id == wanted_exec_id:
                    log['actor_id'] = execution['_id']
                    log['tenant'] = exec_info['tenant']
                    break
    return newList

def updateMongoPermissions(old_json):
    newList = []
    for actor in old_json:
        aid = actor['_id']
        newDict = {'_id': aid}
        newDict.update(actor[aid])
        newList.append(newDict)
    return newList

def updateMongoExecutions(old_json):
    newList = []
    for actor in old_json:
        aid = actor['_id']
        newDict = {'_id': aid}
        newDict.update(actor[aid])
        newList.append(newDict)
    return newList

def updateMongoClients(old_json):
    newList = []
    for actor in old_json:
        aid = actor['_id']
        newDict = {'_id': aid}
        newDict.update(actor[aid])
        newList.append(newDict)
    return newList

# Redis Conversion
# Helper function that converts a Redis DB into a JSON dict
def redis2dict(db):
    allDocs = []
    redisDB = redis.Redis(db=db, port=6397)
    for key in redisDB.scan_iter():
        key = key.decode('utf-8')
        jsonDict = json.loads(redisDB.get(key))
        jsonDict['_id'] = key
        allDocs.append(jsonDict)
    return allDocs

### Grabbing old data

In [417]:
old_logs_store_json = list(db['1'].find({}))
old_permissions_store_json = list(db['2'].find({}))
old_executions_store_json = list(db['3'].find({}))
old_clients_store_json = list(db['4'].find({}))
old_actors_store_json = redis2dict('1')
old_workers_store_json = redis2dict('2')
old_nonce_store_json = redis2dict('3')
old_alias_store_json = redis2dict('4')
old_pregen_clients_json = redis2dict('5')

### The converting

In [418]:
converted_permissions = updateMongoPermissions(old_permissions_store_json)
converted_executions = updateMongoExecutions(old_executions_store_json)
converted_clients = updateMongoClients(old_clients_store_json)
converted_actors = old_actors_store_json
converted_workers = old_workers_store_json
converted_nonces = old_nonce_store_json
converted_aliases = old_alias_store_json
converted_pregen = old_pregen_clients_json
converted_logs = updateMongoLogs(old_logs_store_json, converted_executions)

### The uploading

In [424]:
db_naming = {'n1': converted_logs,
             'n2': converted_permissions,
             'n3': converted_executions,
             'n4': converted_clients,
             'n5': converted_actors,
             'n6': converted_workers,
             'n7': converted_nonces,
             'n8': converted_aliases,
             'n9': converted_pregen}

In [425]:
# Convert and upload modified data
for db_name, db_data in db_naming.items(): 
    try:
        print(f"Currently processing db: {db_name}")
        db[db_name].drop()
        db[db_name].insert_many(db_data)
    except TypeError:
        pass

Currently processing db: n1
Currently processing db: n2
Currently processing db: n3
Currently processing db: n4
Currently processing db: n5
Currently processing db: n6
Currently processing db: n7
Currently processing db: n8
Currently processing db: n9


# Comparison Stuff Functions

In [301]:
def issueRaise(new, old, words):
    print(f"Issue - {words}")
    pprint(new)
    print("=======================================")
    pprint(old)
    raise ValueError(words)

def increment(store, key):
    if key in store:
        store[key] += 1
    else:
        store[key] = 1

def incrementVals(store, key, val):
    if key in store:
        store[key].append(val)
    else:
        store[key] = [val]
        
def lenAndValCheck(inpDict):
    try:  
        keyStore = {}
        valStore = {}
        for i, actor in enumerate(inpDict):
                for worker in actor:
                    if worker == '_id':
                        increment(keyStore, '#actor_ids')
                        incrementVals(valStore, '#actor_ids', actor['_id'])
                    else:
                        for key, val in actor[worker].items():
                            increment(keyStore, key)
                            incrementVals(valStore, key, val)
    except AttributeError:
        keyStore = {}
        valStore = {}
        for i, actor in enumerate(inpDict):
                for key, val in actor.items():
                    increment(keyStore, key)
                    incrementVals(valStore, key, val)
    return keyStore, valStore

In [28]:
# Note: Convert old_coll to new stylings and use that as input.
def conversionChecker(new, old, debug=False):
    newKeys, newVals = lenAndValCheck(new)
    oldKeys, oldVals = lenAndValCheck(old)
    if debug:
        print(f'New Keys:\n{newKeys}\n Old Keys:\n{oldKeys}')
    for key in newKeys:
        try:
            if not newKeys[key] == oldKeys[key]:
                issueRaise(newVals[key], oldVals[key], f"Value lengths are not the same. {newKeys[key]} and {oldKeys[key]}.")
        except KeyError:
            raise KeyError(f"Key: {key} not found in old JSON.")
    for key in oldKeys:
        try:
            if not newKeys[key] == oldKeys[key]:
                issueRaise(newVals[key], oldVals[key], f"Value lengths are not the same. {newKeys[key]} and {oldKeys[key]}.")
        except KeyError:
            raise KeyError(f"Key: '{key}' not found in new JSON.")

## New

In [206]:
new_logs_store_json = list(db['1'].find({}))
new_permissions_store_json = list(db['2'].find({}))
new_executions_store_json = list(db['3'].find({}))
new_clients_store_json = list(db['4'].find({}))
new_actors_store_json = list(db['5'].find({}))
new_workers_store_json = list(db['6'].find({}))
new_nonce_store_json = list(db['7'].find({}))
new_alias_store_json = list(db['8'].find({}))
new_pregen_clients_json = list(db['9'].find({}))

## Old

In [ ]:
# Get JSON for each Redis store
old_logs_store_json = list(db['1'].find({}))
old_permissions_store_json = list(db['2'].find({}))
old_executions_store_json = list(db['3'].find({}))
old_clients_store_json = list(db['4'].find({}))
old_actors_store_json = redis2dict('1')
old_workers_store_json = redis2dict('2')
old_nonce_store_json = redis2dict('3')
old_alias_store_json = redis2dict('4')
old_pregen_clients_json = redis2dict('5')

## Comparisons

In [290]:
conversionChecker(new_logs_store_json, updateMongoLogs(old_logs_store_json))

KeyError: "Key: 'exp' not found in new JSON."

In [291]:
conversionChecker(new_permissions_store_json, updateMongoPermissions(old_permissions_store_json))

In [292]:
conversionChecker(new_executions_store_json, updateMongoExecutions(old_executions_store_json))

In [293]:
conversionChecker(new_actors_store_json, old_actors_store_json)

In [294]:
conversionChecker(new_workers_store_json, old_workers_store_json)

In [295]:
conversionChecker(new_nonce_store_json, old_nonce_store_json)

In [296]:
conversionChecker(new_alias_store_json, old_alias_store_json)

In [297]:
conversionChecker(new_pregen_clients_json, old_pregen_clients_json)

# Archive

In [260]:
# Note: Convert old_coll to new stylings and use that as input.
def conversionChecker(new, old):
    if not type(new) == type(old):
        issueRaise(new, old, "Types aren't the same.")
    if isinstance(new, list) and isinstance(old, list):
        if not len(new) == len(old):
            issueRaise(new, old, "Lists aren't the same length.")
        for new_iter, old_iter in zip(new, old):
            conversionChecker(new_iter, old_iter)
    elif isinstance(new, dict) and isinstance(old, dict):
        if not len(new) == len(old):
            issueRaise(new, old, "Dict's aren't the same length.")
        if not new.keys() == old.keys():
            for new_key, old_key in zip(new.keys(), old.keys()):
                if not new_key == old_key:
                    if not len(new_key) == len(old_key):
                        issueRaise(new_key, old_key, "Dict's keys aren't the same.")

        for new_key, old_key in zip(new.keys(), old.keys()):
            if new_key in ['_id', 'exp', 'start_time', 'io', 'runtime', 'cpu', 'FinishedAt', 'StartedAt',
                           'create_time', 'last_update_time', 'message_received_time']:
                pass
            elif new_key in ['actor_id', 'logs', 'id', 'worker_id', 'db_id']:
                if not len(new[new_key]) == len(old[old_key]):
                    issueRaise(new_key, old_key, f"Dict's keys aren't the same. {new[new_key]} | {old[old_key]}")
            else:
                conversionChecker(new[new_key], old[old_key])

    else:
        if not new == old:
            issueRaise(new, old, "Items aren't the same.")